# Implementation of Wide and Deep Learning for Rating Prediction

## Enivronment Setup

After installing the required packages of Neural Collaborative Filtering, we still need to install the `sklearn` package for the Wide and Deep Learning model.

In [1]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from collections import Counter
from itertools import combinations
import random
import matplotlib.pyplot as plt

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

## Data Preprocessing

Below are utility functions that helps us retrieve the numerical values of different features from the dataset, and generate combinations of features to be used by the WDL model.

### Retrieving Continuous Features

In [3]:
def get_continuous_features(df, continuous_columns):
    '''
    params:
        df: input dataframe
        continuous_columns: list of column names of continuous features

    return:
        a numpy array where each row contains the values of continuous features in the corresponding row of the input dataframe
    '''
    continuous_features = df[continuous_columns].values
    return continuous_features

### Categorical Features Cross Product Transformation
This function is used to generate a variety of feature combinations that occurred frequently in the dataset.

For example, the following items occurred frequently in the dataset:

|Item Name|Occurrence|
|---|---|
|A|4|
|B|3|
|C|2|
|D|1|

If we set `topk=3`, it means that we will only consider the top 3 items with the highest occurrence to generate combinations.

If we set `comb_p=2`, it means that we will generate combinations with 2 items in each generated combinations.

In this case, the following combinations will be generated: `[('A', 'B'), ('A', 'C'), ('B', 'C')]`

Test code: `get_top_k_p_combinations(pd.DataFrame({'item_categories': ['A, B, C, D', 'A, B, C', 'A, B', 'A']}), comb_p=2, topk=3, output_freq=False)`

In [4]:
def get_top_k_p_combinations(df, comb_p, topk, output_freq=False):
    '''
    params:
        df: input dataframe
        comb_p: number of elements in each combination (e.g., there are two elements in the combination {fried chicken, chicken and waffle}, and three elements in the combination {fried chicken, chicken and waffle, chicken fried rice})
        topk: number of most frequent combinations to retrieve
        output_freq: whether to return the frequencies of retrieved combinations

    return:
        1. output_freq = True: a list X where each element is a tuple containing a combination tuple and corresponding frequency, and the elements are stored in the descending order of their frequencies
        2. output_freq = False: a list X where each element is a tuple containing a combination tuple, and the elements are stored in the descending order of their frequencies
    '''
    def get_category_combinations(categories_str, comb_p=2):
        categories = categories_str.split(', ')
        return list(combinations(categories, comb_p))
    all_categories_p_combos = df["item_categories"].apply(lambda x: get_category_combinations(x, comb_p)).values.tolist()
    all_categories_p_combos = [tuple(t) for item in all_categories_p_combos for t in item]
    tmp = dict(Counter(all_categories_p_combos))
    sorted_categories_combinations = list(sorted(tmp.items(), key=lambda x: x[1], reverse=True))
    if output_freq:
        return sorted_categories_combinations[:topk]
    else:
        return [t[0] for t in sorted_categories_combinations[:topk]]

### Building Wide Features

In [5]:
def get_wide_features(df, selected_categories_to_idx, top_combinations):
    '''
    params:
        df: input dataframe
        selected_categories_to_idx: a dictionary mapping item categories to corrresponding integral indices
        top_combinations: a list containing retrieved mostly frequent combinantions of item categories

    return:
        a numpy array where each row contains the categorical features' binary encodings and cross product transformations for the corresponding row of the input dataframe
    '''
    def categories_to_binary_output(categories):
        binary_output = [0 for _ in range(len(selected_categories_to_idx))]
        for category in categories.split(', '):
            if category in selected_categories_to_idx:
                binary_output[selected_categories_to_idx[category]] = 1
            else:
                binary_output[0] = 1
        return binary_output
    def categories_cross_transformation(categories):
        current_category_set = set(categories.split(', '))
        corss_transform_output = [0 for _ in range(len(top_combinations))]
        for k, comb_k in enumerate(top_combinations):
            if len(current_category_set & comb_k) == len(comb_k):
                corss_transform_output[k] = 1
            else:
                corss_transform_output[k] = 0
        return corss_transform_output

    category_binary_features = np.array(df.item_categories.apply(lambda x: categories_to_binary_output(x)).values.tolist())
    category_cross_transform_features = np.array(df.item_categories.apply(lambda x: categories_cross_transformation(x)).values.tolist())
    return np.concatenate((category_binary_features, category_cross_transform_features), axis=1)

# Rating Prediction

### Loading train, validation and test data

In [6]:
tr_df = pd.read_csv("data/wd/train.csv")
val_df = pd.read_csv("data/wd/valid.csv")
te_df = pd.read_csv("data/wd/test.csv")

tr_ratings = tr_df["stars"].values
val_ratings = val_df["stars"].values

### Loading content features of users and items

In [7]:
user_df = pd.read_csv("data/wd/user.csv", index_col=0)
item_df = pd.read_csv("data/wd/business.csv", index_col=0)

# Renaming columns by adding prefixes to column names
user_df = user_df.rename(index=str, columns={t: 'user_' + t for t in user_df.columns if t != 'user_id'})
item_df = item_df.rename(index=str, columns={t: 'item_' + t for t in item_df.columns if t != 'business_id'})

In [8]:
user_df

,user_average_stars,user_compliment_cool,user_compliment_cute,user_compliment_funny,user_compliment_hot,user_compliment_list,user_compliment_more,user_compliment_note,user_compliment_photos,user_compliment_plain,...,user_compliment_writer,user_cool,user_elite,user_fans,user_funny,user_name,user_review_count,user_useful,user_id,user_yelping_since
0,3.63,1,0,1,1,0,0,0,0,0,...,0,16,NaN,4,22,Jenna,33,48,88422913727e71e88611fdfe3512fa03,2013-02-21 22:29:06
1,3.48,1,0,1,0,0,1,2,0,2,...,0,22,NaN,3,16,Edie,77,71,e9567f1e494c12c4bed031c792a822d0,2009-10-26 01:00:40
2,3.81,5,0,5,3,0,0,4,0,5,...,3,61,"2016,2017",7,120,Chelsea,55,161,650ee7b86f36dea4f2d95a414334fc00,2008-09-06 02:56:01
3,3.72,206,4,206,121,6,40,103,12,213,...,49,3239,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...",191,2089,Andrew,4652,5947,0f19b7f339e19bfb05acc32ec539103c,2005-07-18 06:22:37
4,3.74,4069,260,4069,3654,57,168,829,1244,1981,...,856,14031,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...",329,11745,Matt,1947,15046,e4fe9ddbbb82eecb31625a70c489822a,2006-06-20 04:28:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4982,3.49,0,0,0,0,0,0,0,0,0,...,0,4,NaN,0,3,James,36,24,d219d73aa034386d8956a0407c3845c1,2012-12-05 05:09:49
4983,3.45,0,0,0,0,0,1,1,1,0,...,2,6,NaN,0,5,Bilal,39,29,a3199188711ea9af718e0786e23684e3,2014-03-22 23:20:38
4984,3.22,4,0,4,0,0,1,2,0,0,...,0,11,2018,2,12,Aaron,25,23,de13364bd51aa0a7eed6bbd107686fc4,2015-11-30 05:26:57
4985,3.44,0,0,0,0,0,0,0,0,1,...,0,5,NaN,0,13,V,19,44,50ed4222cd18d128478b679d51a42390,2012-05-07 14:46:57


In [9]:
item_df

,item_address,item_attributes,business_id,item_categories,item_city,item_hours,item_is_open,item_latitude,item_longitude,item_name,item_postal_code,item_review_count,item_stars,item_state
0,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",23f2cd62b65e6db173f5f40ed9f13a33,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
1,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...",f182f44b05b28d84ad291e4ed2fe5b6a,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
2,6055 E Lake Mead Blvd,"{'BikeParking': 'True', 'BusinessParking': ""{'...",250a1edb90aa01e2c7acbc9acf741b03,"Mexican, Restaurants, Patisserie/Cake Shop, Fo...",Las Vegas,"{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0'...",1,36.195615,-115.040529,Maria's Mexican Restaurant & Bakery,89156,184,4.5,NV
3,"17025 N Scottsdale Rd, Ste 110","{'BikeParking': 'True', 'BusinessParking': ""{'...",2c064aca434048b770215433ab47ce4c,"Desserts, Food, Cupcakes, Bakeries",Scottsdale,"{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ...",1,33.640308,-111.924528,Nothing Bundt Cakes,85255,174,4.0,AZ
4,4606 Penn Ave,"{'CoatCheck': 'False', 'BusinessParking': ""{'g...",8b0c00106eddf28f708ce68938912b7c,"Nightlife, Bars, Polish, Modern European, Rest...",Pittsburgh,"{'Wednesday': '17:0-0:0', 'Thursday': '17:0-0:...",1,40.465694,-79.949324,Apteka,15224,242,4.5,PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10909,"2837 N Power Rd, Ste 102","{'BusinessAcceptsCreditCards': 'True', 'Restau...",87ce9d599fc87e97a4f79f9b279fed77,"Salad, Restaurants, Hot Dogs, American (Tradit...",Mesa,"{'Monday': '10:0-21:30', 'Tuesday': '10:0-21:3...",1,33.467537,-111.682818,Smashburger,85215,151,3.0,AZ
10910,5611 S Valley View Blvd,"{'Caters': 'True', 'RestaurantsTableService': ...",5e2da739e5ce06ae07eed6e5feecfa25,"Farmers Market, Caterers, Food, Street Vendors...",Las Vegas,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:30'...",1,36.087895,-115.190329,Jessie Rae's BBQ,89118,595,4.5,NV
10911,"9719 Sam Furr Rd, Unit C","{'Ambience': ""{'touristy': False, 'hipster': F...",7cb0d2817b37e00198139f2f60895a73,"Pizza, Italian, Restaurants, Seafood",Huntersville,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",1,35.443723,-80.864550,Antico Italian Restaurant,28078,142,4.5,NC
10912,"8164 S. Las Vegas Blvd., #100","{'OutdoorSeating': 'True', 'WiFi': ""u'free'"", ...",5f45e136060325fdd87b9d001e088510,"Food, Coffee & Tea",Las Vegas,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,36.041407,-115.171698,Starbucks,89123,138,3.0,NV


### Expanding the table by using user_id and business_id

Expand the train, validation and test dataset by using `user_id` and `business_id` to query more features from `user_df` and `item_df`.

In [10]:
tr_df = pd.merge(pd.merge(tr_df, user_df, on='user_id'), item_df, on='business_id').reset_index(drop=True)
val_df = pd.merge(pd.merge(val_df, user_df, on='user_id'), item_df, on='business_id').reset_index(drop=True)
te_df = pd.merge(pd.merge(te_df, user_df, on='user_id'), item_df, on='business_id').reset_index(drop=True)

### Preparing continuous features

In [11]:
# Specify the columns containing conitnuous features
continuous_columns = ["user_average_stars", "user_cool", "user_fans", 
                      "user_review_count", "user_useful", "user_funny",
                      "item_is_open", "item_latitude", "item_longitude", 
                      "item_review_count", "item_stars"]

# Get values of continous features for train/validation/test sets using the utility function defined previously

tr_continuous_features = get_continuous_features(tr_df, continuous_columns)
val_continuous_features = get_continuous_features(val_df, continuous_columns)
te_continuous_features = get_continuous_features(te_df, continuous_columns)

# Standardize each feature by removing the mean of the training samples and scaling to unit variance.
# See https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html for more details.
scaler = StandardScaler().fit(tr_continuous_features)

tr_continuous_features = scaler.transform(tr_continuous_features)
val_continuous_features = scaler.transform(val_continuous_features)
te_continuous_features = scaler.transform(te_continuous_features)

In [12]:
tr_continuous_features 


array([[-1.45650354, -0.13715845,  0.37009944, ..., -0.66911938,
         2.89952001,  0.32439266],
       [ 0.9559452 , -0.23879642, -0.31914458, ..., -0.66955713,
        -0.26148944,  1.24924702],
       [ 1.15154915, -0.27727663, -0.37320293, ..., -0.45684728,
         0.08722981,  1.24924702],
       ...,
       [ 0.23873071, -0.26180089, -0.22454246, ...,  1.68149118,
        -0.47380489, -0.60046169],
       [-1.00009432, -0.27727663, -0.35968835, ...,  1.68047494,
        -0.54141373,  0.32439266],
       [-1.82597767, -0.21328236, -0.31914458, ..., -0.45820683,
        -0.06459354,  1.24924702]])

### Preparing deep categorical features

In [13]:
# Sepcify column names of deep categorical features
item_deep_columns = ["item_city", "item_postal_code", "item_state"]

# An array of integers where deep_vocab_lens[i] represents the number of unique values of (i+1)-th deep categorical feature
# Transforming words into indices for each categorical columns
item_deep_vocab_lens = []
for col_name in item_deep_columns:
    # Getting unique values of this deep categorical feature
    unique_values = item_df[col_name].unique()
    
    # Creating a dictionary to map from unique values to the corresponding index
    vocab = dict(zip(unique_values, range(1, len(unique_values)+1)))
    
    # Getting the number of unique values of this deep categorical features
    item_deep_vocab_lens.append(len(vocab)+1)
    
    # Creating a new column where each entry stores the index of this deep categorical feature's value in the same row
    item_df[col_name + "_idx"] = item_df[col_name].apply(lambda x: vocab[x])


# Creating a dictionary mapping each business id to corresponding values of deep categorical features ('business_id' -> ['item_city_idx', 'item_postal_code_idx', 'item_state_idx'] in this case)
item_deep_idx_columns = [t + "_idx" for t in item_deep_columns]
item_to_deep_categorical_features = dict(zip(item_df['business_id'].values, item_df[item_deep_idx_columns].values.tolist()))

# Creating numpy arrays storing corresponding deep categorical features' values of train/validation/test sets using the above mapping
tr_deep_categorical_features = np.array(tr_df['business_id'].apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())
val_deep_categorical_features = np.array(val_df['business_id'].apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())
te_deep_categorical_features = np.array(te_df['business_id'].apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())

In [14]:
item_to_deep_categorical_features

{'23f2cd62b65e6db173f5f40ed9f13a33': [1, 1, 1],
 'f182f44b05b28d84ad291e4ed2fe5b6a': [2, 2, 2],
 '250a1edb90aa01e2c7acbc9acf741b03': [3, 3, 3],
 '2c064aca434048b770215433ab47ce4c': [4, 4, 4],
 '8b0c00106eddf28f708ce68938912b7c': [5, 5, 5],
 '76af52fd0698cdd4aa5ed82b2e8d255c': [6, 6, 4],
 '1f17c4f965038e52a6cbae0dae1009f6': [3, 7, 3],
 'f46ed23717ea97439cb716fe5cfa6d44': [7, 8, 4],
 'd40d36158849e366378f392ff34b03d2': [8, 9, 3],
 '350dcea425a8bba0f4077f10204d1ce5': [2, 10, 2],
 '109ec8201ca31675ace1d83244c3413b': [9, 11, 1],
 '66fd22ff80aefcfabc6a6e78f8ce7217': [4, 12, 4],
 '2f3252bed7577515a828d1e773da225c': [3, 13, 3],
 '14b53daf40828fd98d84bf400611d38b': [3, 14, 3],
 'e0da1f0dec448e6ebe5b1ef7fa523e3c': [4, 12, 4],
 '0aaf020243eb984245437733cc8c95fa': [9, 15, 1],
 '1194284c5ada92cf177bfd21e87ef082': [3, 16, 3],
 '4786632697784b614806ded7078c4e34': [10, 17, 6],
 '83d43e7340dc20f2577783e3ad59d8ac': [6, 18, 4],
 '99913c55bb283c9708a339414791afb6': [3, 13, 3],
 '42becccc017faeede2b5142d83

In [24]:
tr_deep_categorical_features

array([[  3,  13,   3],
       [  3,  44,   3],
       [  6,  18,   4],
       ...,
       [  2, 118,   2],
       [  2,  85,   2],
       [  6, 111,   4]])

### Preparing wide features

Preparing binary encoding for each selected category

In [15]:
# Collect the categories of all items 
all_categories = [category for category_list in item_df.item_categories.values for category in category_list.split(", ")]

# Sort all unique values of the item categories by their frequencies in descending order
category_sorted = sorted(Counter(all_categories).items(), key=lambda x: x[1], reverse=True)

# Select top 500 most frequent categories
selected_categories = [t[0] for t in category_sorted[:500]]

# Create a dictionary mapping each secleted category to a unique integral index
selected_categories_to_idx = dict(zip(selected_categories, range(1, len(selected_categories) + 1)))

# Map all categories unseen in the item df to index 0
selected_categories_to_idx['unk'] = 0

# Create a dictionary mapping each integral index to corresponding category
idx_to_selected_categories = {val: key for key, val in selected_categories_to_idx.items()}

Preparing cross product transformation for categories

In [16]:
# Get most frequent categories combinantions using the utility function defined previously and store them in the folloing list
top_combinations = []

# Get top 50 most frequent two-categories combinantions in the train set

top_combinations += get_top_k_p_combinations(tr_df, 2, 50, output_freq=False)

# Get top 30 most frequent three-categories combinantions in the train set
top_combinations += get_top_k_p_combinations(tr_df, 3, 30, output_freq=False)

# Get top 20 most frequent four-categories combinantions in the train set
top_combinations += get_top_k_p_combinations(tr_df, 4, 20, output_freq=False)

# Convert each combinantion in the list to a set data structure
top_combinations = [set(t) for t in top_combinations]

In [17]:
# Getting values of wide features for train/validation/test sets using the utility function defined previously
# The following matrices should have a shape of (n_samples, len(selected_categories_to_idx)+len(top_combinations))
tr_wide_features = get_wide_features(tr_df, selected_categories_to_idx, top_combinations)
val_wide_features = get_wide_features(val_df, selected_categories_to_idx, top_combinations)
te_wide_features = get_wide_features(te_df, selected_categories_to_idx, top_combinations)

### Concatenating continuous features, deep categorical features and wide features as an input list

In [18]:
tr_features = []
tr_features.append(tr_continuous_features)
tr_features += [tr_deep_categorical_features[:, i] for i in range(tr_deep_categorical_features.shape[1])]
tr_features.append(tr_wide_features)

val_features = []
val_features.append(val_continuous_features)
val_features += [val_deep_categorical_features[:, i] for i in range(val_deep_categorical_features.shape[1])]
val_features.append(val_wide_features)

te_features = []
te_features.append(te_continuous_features)
te_features += [te_deep_categorical_features[:, i] for i in range(te_deep_categorical_features.shape[1])]
te_features.append(te_wide_features)

## Model Implementation

First, we will implement a custom dataset class for the rating prediction task. 

In [19]:
class RatingDataset(Dataset):
    def __init__(self, features, ratings):
        self.continuous = torch.FloatTensor(features[0])
        self.deep_categorical = [torch.LongTensor(f) for f in features[1:-1]]
        self.wide = torch.FloatTensor(features[-1])
        self.ratings = torch.FloatTensor(ratings)

    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        return (
            self.continuous[idx],
            *[dc[idx] for dc in self.deep_categorical],
            self.wide[idx],
            self.ratings[idx]
        )

### Implementation of the Wide and Deep Learning model

In [20]:
class WideDeepModel(nn.Module):
    def __init__(self, len_continuous, deep_vocab_lens, len_wide, embed_size):
        super().__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(vocab_size, embed_size) for vocab_size in deep_vocab_lens
        ])

        self.dnn = nn.Sequential(
            nn.Linear(len_continuous + len(deep_vocab_lens) * embed_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        self.fc = nn.Linear(64 + len_wide, 1)

    def forward(self, continuous, *args):
        deep_categorical = args[:len(self.embeddings)]
        wide = args[-1]

        embeds = []
        for i, emb_layer in enumerate(self.embeddings):
            embeds.append(emb_layer(deep_categorical[i]))
        embeds = torch.cat(embeds, 1)

        deep_input = torch.cat([continuous, embeds], dim=1)
        dnn_output = self.dnn(deep_input)

        combined = torch.cat([dnn_output, wide], dim=1)
        return self.fc(combined).squeeze()


## Training the model

In [21]:
train_dataset = RatingDataset(tr_features, tr_ratings)
val_dataset = RatingDataset(val_features, val_ratings)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device", device)

model = WideDeepModel(
    len_continuous=len(continuous_columns),
    deep_vocab_lens=item_deep_vocab_lens,
    len_wide=tr_wide_features.shape[1],
    embed_size=100
)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 3
train_losses = []
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for continuous, city, postal_code, state, wide, ratings in train_loader:
        optimizer.zero_grad()
        outputs = model(
            continuous.to(device),
            city.to(device),
            postal_code.to(device),
            state.to(device),
            wide.to(device)
        )
        loss = criterion(outputs, ratings.to(device))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    train_losses.append(epoch_loss / len(train_loader))
    print(f"Epoch {epoch+1} loss: {train_losses[-1]:.4f}")

using device cpu
Epoch 1 loss: 1.4332
Epoch 2 loss: 1.2353
Epoch 3 loss: 1.2646


## Evaluation on the validation set

In [22]:
def compute_rmse(model, loader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for continuous, city, postal_code, state, wide, ratings in loader:
            outputs = model(
                continuous.to(device),
                city.to(device),
                postal_code.to(device),
                state.to(device),
                wide.to(device)
            )
            total_loss += nn.MSELoss(reduction='sum')(outputs, ratings.to(device)).item()
    mse = total_loss / len(loader.dataset)
    return torch.sqrt(torch.tensor(mse)).item()

train_rmse = compute_rmse(model, train_loader)
val_rmse = compute_rmse(model, val_loader)

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")

Train RMSE: 1.0867
Validation RMSE: 1.0934
